In [1]:
#import sys # This is not needed on your machine. Please comment out.
#sys.path.append("C:/Users/wassimj/Documents/Github") # This is not needed on your machine. Please comment out.

## Import the topologicpy classes

In [2]:
import topologicpy
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Plotly import Plotly
print ("current version is: "+ topologicpy.__version__)

current version is: 0.5.2


## Create the basic building geometry and doors

In [3]:
r1 = Cell.Prism(origin=Vertex.ByCoordinates(0,4.5,0), width=3, length=7.5, height=3, placement="lowerleft")
r2 = Cell.Prism(origin=Vertex.ByCoordinates(0,0,0), width=3, length=4.5, height=3, placement="lowerleft")
r3 = Cell.Prism(origin=Vertex.ByCoordinates(3,7.5,0), width=3, length=4.5, height=3, placement="lowerleft")
r4 = Cell.Prism(origin=Vertex.ByCoordinates(3,0,0), width=3, length=7.5, height=3, placement="lowerleft")
r5 = Cell.Prism(origin=Vertex.ByCoordinates(6,6,0), width=3, length=6, height=3, placement="lowerleft")
r6 = Cell.Prism(origin=Vertex.ByCoordinates(6,0,0), width=3, length=6, height=3, placement="lowerleft")
r7 = Cell.Prism(origin=Vertex.ByCoordinates(9,7.5,0), width=3, length=4.5, height=3, placement="lowerleft")
r8 = Cell.Prism(origin=Vertex.ByCoordinates(9,3,0), width=3, length=4.5, height=3, placement="lowerleft")
r9 = Cell.Prism(origin=Vertex.ByCoordinates(12,3,0), width=3, length=9, height=3, placement="lowerleft")
r10 = Cell.Prism(origin=Vertex.ByCoordinates(9,0,0), width=6, length=3, height=3, placement="lowerleft")
doorA = Face.Rectangle(width=0.9, length=2.1, placement="lowerleft")
doorA = Topology.Rotate(doorA, x=1, y=0, z=0, degree=90)
doorA = Topology.Rotate(doorA, x=0, y=0, z=1, degree=90)
doorA = Topology.Translate(doorA, x=0, y=-0.45, z=0.05)
doorB = Topology.Rotate(doorA, x=0, y=0, z=1, degree=90)
d1 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(3, 6, 0))
d2 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(6, 3, 0))
d3 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(6, 6.75, 0))
d4 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(6, 9, 0))
d5 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(9, 4.5, 0))
d6 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(9, 6.75, 0))
d7 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(9, 9, 0))
d8 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(12, 6, 0))
d9 = Topology.Place(doorA, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(12, 9, 0))
d10 = Topology.Place(doorB, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(1.5, 4.5, 0))
d11 = Topology.Place(doorB, originA=Vertex.Origin(), originB=Vertex.ByCoordinates(13.5, 3, 0))
Topology.Show(r1,r2,r3,r4,r5,r6,r7,r8,r9,r10,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11, renderer="browser")

## Create the CellComplex from the rooms and add doors as apertures

In [4]:
building = CellComplex.ByCells([r1,r2,r3,r4,r5,r6,r7,r8,r9,r10])
doors = [d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11]
building = Topology.AddApertures(building, doors, subTopologyType="face", exclusive=True)
Topology.Show(building, doors,renderer="browser")

## Create the basic dual graph to check accuracy of modelling and topology

In [12]:
g = Graph.ByTopology(building, direct=False, directApertures=True)
g_top = Graph.Topology(g)
data01 = Plotly.DataByTopology(g_top, vertexSize=5, edgeWidth=3, edgeColor="red")
data02 = Plotly.DataByTopology(building)
data03 = Plotly.DataByTopology(Cluster.ByTopologies(doors))
figure = Plotly.FigureByData(data01+data02+data03)
Plotly.Show(figure,renderer="browser")

## Calculate the PageRank of the created graph. Assume it is un-directed (bidirectional edges)

In [7]:
page_rank = Graph.PageRank(g, directed=False, maxIterations=100, normalize=False)
page_rank = [round(x, 2) for x in page_rank]
print(page_rank)

[0.13, 0.1, 0.06, 0.13, 0.13, 0.09, 0.05, 0.05, 0.17, 0.09]


## Transfer the PageRank values to vertices to apply to the rooms for colour-coding

In [8]:
vertices = Graph.Vertices(g)
selectors = []
for i, v in enumerate(vertices):
    d = Dictionary.ByKeyValue("pageRank", page_rank[i])
    s = Vertex.ByCoordinates(Vertex.X(v), Vertex.Y(v), 0) #Move to the ground to apply to room floor at z=0
    s = Topology.SetDictionary(s, d)
    selectors.append(s)
    print(Vertex.Coordinates(v), page_rank[i])

[10.5, 5.25, 1.5] 0.13
[1.5, 8.25, 1.5] 0.1
[1.5, 2.25, 1.5] 0.06
[4.5, 3.75, 1.5] 0.13
[13.5, 7.5, 1.5] 0.13
[7.5, 3.0, 1.5] 0.09
[12.0, 1.5, 1.5] 0.05
[4.5, 9.75, 1.5] 0.05
[7.5, 9.0, 1.5] 0.17
[10.5, 9.75, 1.5] 0.09


## Get the floor of each room and transfer the dictionaries.

In [9]:
dictionary = CellComplex.Decompose(building)
bhf = dictionary['bottomHorizontalFaces']
shell = Shell.ByFaces(bhf)
shell = Topology.TransferDictionariesBySelectors(shell, selectors=selectors, tranFaces=True)
edges = Topology.Edges(building)
for door in doors:
    edges += Topology.Edges(door)

## Hover over each room floor to see its PageRank value

In [11]:
Topology.Show(shell, edges,renderer="browser",faceLabelKey="pageRank", faceGroupKey="pageRank", faceGroups=page_rank, colorScale="jet", faceOpacity=1, backgroundColor="white")